# Описание задачи
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.


Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Описание данных
**Таблица** books <br>
Содержит данные о книгах:<br>
book_id — идентификатор книги;<br>
author_id — идентификатор автора;<br>
title — название книги;<br>
num_pages — количество страниц;<br>
publication_date — дата публикации книги;<br>
publisher_id — идентификатор издателя.<br>
<br>
**Таблица** authors<br>
Содержит данные об авторах:<br>
author_id — идентификатор автора;<br>
author — имя автора.<br>
Таблица publishers<br>
Содержит данные об издательствах:<br>
publisher_id — идентификатор издательства;<br>
publisher — название издательства;<br>
<br>
**Таблица** ratings<br>
Содержит данные о пользовательских оценках книг:<br>
rating_id — идентификатор оценки;<br>
book_id — идентификатор книги;<br>
username — имя пользователя, оставившего оценку;<br>
rating — оценка книги.<br>
<br>
**Таблица** reviews<br>
Содержит данные о пользовательских обзорах на книги:<br>
review_id — идентификатор обзора;<br>
book_id — идентификатор книги;<br>
username — имя пользователя, написавшего обзор;<br>
text — текст обзора.

# Задача
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще
50 страниц — так мы исключим из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг — учитывать
только книги с 50 и более оценками;
* Посчитать среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

# Знакомство с данными

In [6]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
#Открываем соединение, получаем данные из sql, закрываем соединение'''
   with engine.connect() as con:
      return pd.read_sql(sql=sa.text(query), con = con)
# формируем запрос и выводим данные

for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = '''SELECT * FROM {} LIMIT 5'''.format(table)
    data = get_sql_data(query)
    print(f'НАЗВАНИЕ ТАБЛИЦЫ: {table}', '\n')
    print(data, '\n')

НАЗВАНИЕ ТАБЛИЦЫ: books 

   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268   

НАЗВАНИЕ ТАБЛИЦЫ: authors 

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   

# Посчитаем, сколько книг вышло после 1 января 2000 года:

In [31]:
query = '''SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
data = get_sql_data(query)
data

,count
0,819


**После 1 января 2000 года вышло 819 книг.**

# Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [47]:
query = '''SELECT b.book_id, title, COUNT(review_id) as reviews_number, AVG(rating) as average_rating
FROM books AS b
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
GROUP BY b.book_id, title
ORDER BY AVG(rating) DESC'''
data = get_sql_data(query)
data

,book_id,title,reviews_number,average_rating
0,518,Pop Goes the Weasel (Alex Cross #5),4,5.00
1,732,The Ghost Map: The Story of London's Most Terr...,4,5.00
2,347,In the Hand of the Goddess (Song of the Liones...,6,5.00
3,610,Tai-Pan (Asian Saga #2),4,5.00
4,330,How to Be a Domestic Goddess: Baking and the A...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,12,2.25
996,371,Junky,4,2.00
997,316,His Excellency: George Washington,4,2.00
998,202,Drowning Ruth,9,2.00


**Для каждой книги выведено название, количество обзоров и средний рейтинг. Рейтинг изменяется от 1,5 до 5.**

# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры:

In [64]:
query = '''SELECT COUNT(book_id) as quantity_of_books, b.publisher_id, publisher
FROM books AS b
LEFT JOIN publishers AS pub ON b.publisher_id = pub.publisher_id
WHERE num_pages > 50
GROUP BY  b.publisher_id, publisher
ORDER BY quantity_of_books DESC LIMIT 1'''
data = get_sql_data(query)
data

,quantity_of_books,publisher_id,publisher
0,42,212,Penguin Books


**Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books, а именно 42.**

# Определим автора с самой высокой средней оценкой книг — учитывать будем только книги с 50 и более оценками:

In [86]:
query ='''SELECT boo.author, AVG(rating)
          FROM (SELECT b.book_id as book_id, a.author AS author
                FROM books AS b
                JOIN ratings AS r ON r.book_id = b.book_id
                LEFT JOIN authors as a ON b.author_id =  a.author_id
                GROUP BY b.book_id,a.author_id
                HAVING COUNT(rating_id) >= 50)  as boo
          JOIN ratings AS r ON r.book_id = boo.book_id
          GROUP BY boo.author
          ORDER BY AVG(rating) DESC LIMIT(1)'''
data = get_sql_data(query)
data

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


**Автором с самой высокой средней оценкой книг среди книг с 50 и более оценками является J.K. Rowling/Mary GrandPré со средней оценкой 4.287097.**

# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [91]:
query ='''SELECT AVG(c)
          FROM (SELECT COUNT(review_id) AS c
                FROM (SELECT username
                      FROM ratings
                      GROUP BY username
                      HAVING COUNT(rating_id) > 48)  as us
                JOIN reviews AS r ON r.username = us.username
                GROUP BY r.username) AS S'''
data = get_sql_data(query)
data

,avg
0,24.0


**Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, - 24 обзора.**

# Выводы:
**1. После 1 января 2000 года вышло 819 книг;**<br>
**2. Для каждой книги выведено название, количество обзоров и посчитан средний рейтинг. Рейтинг изменяется от 1,5 до 5;**<br>
**3. Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books, а именно 42;**<br>
**4. Автором с самой высокой средней оценкой книг среди книг с 50 и более оценками является J.K. Rowling/Mary GrandPré со**<br>**средней оценкой 4.287097;**<br>
**5. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, - 24 обзора.**<br>